## Parameters

In [59]:
wg_height=220e-3
wg_width=450e-3
etch_depth=160e-3
wavelength=1550e-3
domain_width=3
box_height=2
tox_height=1
length_strip=1
domain_height=box_height+tox_height
length_taper=3
length_rib=1
width_rib=2
straight_length1=0.872
straight_length2=5.8579

In [60]:
import nazca as nd
import nazca.interconnects as IC
import numpy as np

# Define slab and waveguide layers
nd.add_layer2xsection(xsection='slabXS', layer=10, accuracy=0.001)
nd.add_layer2xsection(xsection='waveguideXS', layer=20, accuracy=0.001)

# Create interconnects
icSlab = IC.Interconnect(width=0.45, radius=10, xs='slabXS')
icWaveguide = IC.Interconnect(width=0.45, radius=10, xs='waveguideXS')  # New interconnect on layer 20

# Define slab taper function
def slabTaper(length=3.0, ribWidth=0.49, slabWidth=2.0):
    with nd.Cell(name='SlabTaper_{}_{}'.format(length, slabWidth)) as bb:
        # Place Taper
        chTaper = icSlab.taper(length=length, width1=ribWidth, width2=slabWidth)
        chTaper.put(0, 0)
        
        # Put Pins
        nd.Pin('a0', pin=chTaper.pin['a0']).put()
        nd.Pin('b0', pin=chTaper.pin['b0']).put()
        nd.put_stub([], length=0)
        nd.put_stub(['a0', 'b0'], length=0)

    return bb

def y_splitter(x, y, width1=0.45, length=5.0):
    """Creates a Y-splitter from one input to two outputs."""
    with nd.Cell(name="Y_Splitter") as ysplit:
        # Define input waveguide
        in_wg = icWaveguide.strt(length=length, width=width1)
        input = in_wg.put(x, y)
        
        # Define output branches
        out_wg1 = icWaveguide.bend(radius=5, angle=-45)  # First branch
        out_wg2 = icWaveguide.bend(radius=5, angle=45)   # Second branch
        
        output1 = out_wg1.put(input.pin["b0"])  # Place first output
        output2 = out_wg2.put(input.pin["b0"])  # Place second output
        
        # Pins for connectivity
        nd.Pin("input", pin=input.pin["a0"]).put()
        nd.Pin("output1", pin=output1.pin["b0"]).put()
        nd.Pin("output2", pin=output2.pin["b0"]).put()
        
    return ysplit


# Function to create a bend with a starting angle
def custom_bend(x, y, start_angle, bend_radius, bend_angle):
    """Creates a bend with customizable parameters.

    Args:
        x, y (float): Starting position.
        start_angle (float): Initial orientation before bending.
        bend_radius (float): Radius of the bend.
        bend_angle (float): Bend angle in degrees.

    Returns:
        Cell: The bend element.
    """
    with nd.Cell(name=f"Bend_{bend_radius}_{bend_angle}") as bend_cell:
        bend = icWaveguide.bend(radius=bend_radius, angle=bend_angle)
        p_bend=bend.put(x, y, start_angle)  # Apply starting angle
        nd.Pin("b0", pin=p_bend.pin["b0"]).put()
    return bend_cell


# Function to create a straight waveguide with a starting angle
def straight_waveguide(x, y, start_angle, width, length):
    """Creates a straight waveguide with defined width, length, and orientation.

    Args:
        x, y (float): Starting position.
        start_angle (float): Initial orientation before placing.
        width (float): Width of the waveguide.
        length (float): Length of the waveguide.

    Returns:
        Cell: The straight waveguide element.
    """
    with nd.Cell(name=f"StraightWG_{length}_{width}") as wg_cell:
        wg = icWaveguide.strt(length=length, width=width)
        p_wg=wg.put(x, y, start_angle)  # Apply initial orientation
        nd.Pin("b0", pin=p_wg.pin["b0"]).put()
    return wg_cell


# Function to create a directional coupler
def directional_coupler(x, y, length=1.2, gap=50e-3):
    """Creates a directional coupler with two parallel waveguides.

    Args:
        x, y (float): Start position.
        length (float): Length of the waveguides in microns.
        gap (float): Distance between waveguides in microns.

    Returns:
        Cell: The directional coupler element.
    """
    with nd.Cell(name=f"Directional_Coupler_{length}_{gap}") as dc_cell:
        # First waveguide
        wg1 = icWaveguide.strt(length=length, width=0.45)
        pwg1=wg1.put(x, y)

        # Second waveguide, placed with a gap
        wg2 = icWaveguide.strt(length=length, width=0.45)
        pwg2=wg2.put(x, y + gap+0.45)  # Offset by the gap distance

        input_wg1 = icWaveguide.bend(radius=5, angle=45)  # First branch
        input_wg2 = icWaveguide.bend(radius=5, angle=-45)   # Second branch
        
        input1 = input_wg1.put(pwg1.pin["a0"])  # Place first output
        input2 = input_wg2.put(pwg2.pin["a0"]) 

        out_wg1 = icWaveguide.bend(radius=5, angle=-45)  # First branch
        out_wg2 = icWaveguide.bend(radius=5, angle=45)   # Second branch
        
        output1 = out_wg1.put(pwg1.pin["b0"])  # Place first output
        output2 = out_wg2.put(pwg2.pin["b0"]) 

        # Define pins for connectivity
        nd.Pin("input1", pin=input1.pin["b0"]).put()
        nd.Pin("input2", pin=input2.pin["b0"]).put()
        nd.Pin("output1", pin=output1.pin["b0"]).put()
        nd.Pin("output2", pin=output2.pin["b0"]).put()

    return dc_cell

def laser_system(length, length2):
    """Creates a full laser system with defined inputs/outputs."""
    with nd.Cell(name=f"laser_system_{length}") as laser_cell:
        bend = icWaveguide.bend(radius=5, angle=-45)
        pbend1=bend.put()
        in_wg = icWaveguide.strt(length=length, width=0.45)
        input = in_wg.put(pbend1.pin["b0"])
        
        # Define output branches
        out_wg1 = icWaveguide.bend(radius=5, angle=-45)  # First branch
        out_wg2 = icWaveguide.bend(radius=5, angle=45)   # Second branch
        
        output1 = out_wg1.put(input.pin["b0"])  # Place first output
        output2 = out_wg2.put(input.pin["b0"])  # Place second output

        # Lower branch of laser
        wg1 = icWaveguide.strt(length=length, width=0.45)
        pwg1=wg1.put(output1.pin["b0"])
        bend = icWaveguide.bend(radius=5, angle=45)
        pbend1=bend.put(pwg1.pin["b0"])
        wg2=icWaveguide.strt(length=length2, width=0.45)
        pwg2=wg2.put(pbend1.pin["b0"])

        bend3=icWaveguide.bend(radius=5, angle=45)
        pbend2=bend3.put(output2.pin["b0"])

        # Assign proper connectivity pins
        nd.Pin("input", pin=pbend1.pin["a0"]).put()
        nd.Pin("output1", pin=pbend2.pin["b0"]).put()
        nd.Pin("output2", pin=pwg2.pin["b0"]).put()

    return laser_cell




# Create tapers using correct interconnects
taper1 = slabTaper(length=length_taper, ribWidth=wg_width, slabWidth=width_rib)
taper2 = icSlab.taper(length=length_taper, width1=wg_width, width2=width_rib)



length_str1=10
length_str2=1
length_str3=12.8
def run_until_perfect():
    distance=1
    length_str3=12.8
    while (distance>0.0001):
        p_ysplit2 = y_splitter(0,5).put(0, 0) #INPUT
        bend = custom_bend(x=0, y=0, start_angle=0, bend_radius=5, bend_angle=-45)
                #s for signal
        p_bend_sbelow=bend.put(p_ysplit2.pin["output1"]) #lower branch of signal
        bend = custom_bend(x=0, y=0, start_angle=0, bend_radius=5, bend_angle=45) 
        p_bend_sabove=bend.put(p_ysplit2.pin["output2"]) #upper branch of signal

                #lower branch
        straight1=straight_waveguide(0, 0, 0, 0.45, length_str1)
        lengthtemp1=1 + 0.136698125872325 ##############
        straight_interconnect1=icWaveguide.strt(length=lengthtemp1, width=0.45)
        pstraight_interconnect1=straight_interconnect1.put(p_bend_sabove.pin["b0"])

        lengthtemp2=1.5 #################
        taper2 = icWaveguide.taper(length=lengthtemp2, width1=wg_width, width2=1)
        ptaperinter_1=taper2.put(pstraight_interconnect1.pin["b0"])
        coord1=ptaperinter_1.pin["b0"]
        lengthinter=1 ##################
        p_block=icWaveguide.strt(length=1, width=1).put(ptaperinter_1.pin["b0"])
        p_taperinter_3=icWaveguide.taper(length=lengthtemp2, width1=1, width2=0.45).put(p_block.pin["b0"])

        p_str_sabove=icWaveguide.strt(length=length_str1-lengthinter-2*lengthtemp2-lengthtemp1, width=0.45).put(p_taperinter_3.pin["b0"])


        p_str_sbelow=straight1.put(p_bend_sbelow.pin["b0"])
        pbend_slowest=custom_bend(0, 0, 0, 5, 90).put(p_str_sbelow.pin["b0"])

        straight2=straight_waveguide(0, 0, 0, 0.45, length_str2)

        p_str_slow=straight2.put(pbend_slowest.pin["b0"])
        p_bend_sibldc=custom_bend(0, 0, 0, 5, 45).put(p_str_slow.pin["b0"])

        straight3=straight_waveguide(0, 0, 0, 0.45, length_str3)




                #signal above the y beam splitter
        bend = custom_bend(x=0, y=0, start_angle=0, bend_radius=5, bend_angle=-90) 
        p_bend_sabvstr=bend.put(p_str_sabove.pin["b0"])

        p_str1=straight2.put(p_bend_sabvstr.pin["b0"])
        bend=custom_bend(0, 0, 0, 5, -45)
        p_bend_si_dc=bend.put(p_str1.pin["b0"])
        p_strabove1=straight3.put(p_bend_si_dc.pin["b0"])

                # Place directional coupler at the waveguide's output port
        dc = directional_coupler(p_strabove1.pin["b0"].x, p_strabove1.pin["b0"].y)
        pdc1=dc.put("input2")

        dc_input1_pin = pdc1.pin["input1"]  # Fetch the exact input pin of the DC
        laser_system_cell = laser_system(straight_length1, straight_length2)
        output_pin = laser_system_cell.pin["output1"]
        x_shift = dc_input1_pin.x - output_pin.x
        y_shift = dc_input1_pin.y - output_pin.y
        plsystem=laser_system_cell.put(x_shift, y_shift)
        pinterconnect3=icWaveguide.strt(length=4.5+0.863301874127664, width=0.45).put(plsystem.pin["a0"])
        ptaperinter2=icWaveguide.taper(length=lengthtemp2, width1=0.45, width2=1).put(pinterconnect3.pin["b0"])
        coord2=ptaperinter2.pin["b0"]

        ptaperinter4=icWaveguide.taper(length=lengthtemp2, width1=0.45, width2=1).put(coord1.x-0.5-lengthtemp2, coord1.y+0.5)
        length_equal1=ptaperinter4.pin["a0"].x-p_ysplit2.pin["a0"].x
        pstrteq=icWaveguide.strt(length=length_equal1, width=0.45).put(ptaperinter4.pin["a0"])

                #second directional coupler
        p_afterbandstrslow=straight3.put(p_bend_sibldc.pin["b0"])
        dc = directional_coupler(p_afterbandstrslow.pin["b0"].x, p_afterbandstrslow.pin["b0"].y)
        pdc2=dc.put("input1")

        p1=np.array([pdc2.pin["input2"].x, pdc2.pin["input2"].y])
        p2=np.array([plsystem.pin["output2"].x, plsystem.pin["output2"].y])

        distance=np.sqrt(np.sum((p1 - p2)**2))
        need_to_move=distance/2
        length_str3=length_str3+need_to_move
        print(distance)

        pbendafdc1=icWaveguide.bend(5, 45).put(pdc1.pin["output1"])
        pbendafdc2=icWaveguide.bend(5, -45).put(pdc1.pin["output2"])
        pbendafdc3=icWaveguide.bend(5, 45).put(pdc2.pin["output1"])
        pbendafdc4=icWaveguide.bend(5, -45).put(pdc2.pin["output2"])
        length_str4=20
        straight4=straight_waveguide(0, 0, 0, 0.45, length_str4)
        pstr_befPD1=straight4.put(pbendafdc1.pin["b0"])
        pstr_befPD2=straight4.put(pbendafdc2.pin["b0"])
        pstr_befPD3=straight4.put(pbendafdc3.pin["b0"])
        pstr_befPD4=straight4.put(pbendafdc4.pin["b0"])
        
        taper2 = icSlab.taper(length=length_taper, width1=wg_width, width2=width_rib)
        taper1.put(pstr_befPD1.pin["b0"])
        icWaveguide.strt(length_taper, 0.45).put(pstr_befPD1.pin["b0"])
        taper2.put(pstr_befPD2.pin["b0"])
        icWaveguide.strt(length_taper, 0.45).put(pstr_befPD2.pin["b0"])
        taper2.put(pstr_befPD3.pin["b0"])
        icWaveguide.strt(length_taper, 0.45).put(pstr_befPD3.pin["b0"])
        taper2.put(pstr_befPD4.pin["b0"])
        icWaveguide.strt(length_taper, 0.45).put(pstr_befPD4.pin["b0"])

        # Export GDS
        print("length 3= ", length_str3)
        nd.export_gds(filename='tapers.gds')

In [61]:
run_until_perfect()

0.02604122402573999
length 3=  12.81302061201287
Starting layout export...
...gds generation
...Wrote file './tapers.gds'


0.0076273273392770775
length 3=  12.81683427568251
Starting layout export...
...gds generation
...Wrote file './tapers.gds'


0.002234092873798735
length 3=  12.81795132211941
Starting layout export...
...gds generation
...Wrote file './tapers.gds'


0.0006546934154742119
length 3=  12.818278668827146
Starting layout export...
...gds generation
...Wrote file './tapers.gds'


0.00019292206614505262
length 3=  12.818375129860218
Starting layout export...
...gds generation
...Wrote file './tapers.gds'


6.036128445449019e-05
length 3=  12.818405310502445
Starting layout export...
...gds generation
...Wrote file './tapers.gds'
